In [1]:
import pandas as pd
import time
start_time = time.time()

In [2]:
print(pd.__version__)

1.1.1


In [3]:
# Load beneficiary, outpatient, and carrier claims files
ben_s1 = pd.read_csv("data\DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv", low_memory = False)
car_a_s1 = pd.read_csv("data\DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.csv", low_memory = False)
countyxwalk = pd.read_csv("gis_data\ssa_fips_state_county2011.csv", low_memory = False)

In [4]:
# Pull records for beneficiaries in state of interest with Part B coverage
# SSA state code 31 is NJ
ben_s1_state = ben_s1[(ben_s1.SP_STATE_CODE == 31) & (ben_s1.BENE_SMI_CVRAGE_TOT_MONS > 0)]

# Left join carrier claims to beneficiary data, to only retain claims associated with 2008 NJ beneficiary
ben_s1_state_cara = pd.merge(ben_s1_state,car_a_s1,on='DESYNPUF_ID',how='left')

In [5]:
ben_s1_state_cara.info() #74,056 rows (NJ, Part B, car_a claims files sample 1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74056 entries, 0 to 74055
Columns: 173 entries, DESYNPUF_ID to LINE_ICD9_DGNS_CD_13
dtypes: float64(104), int64(20), object(49)
memory usage: 98.3+ MB


In [6]:
# add claims year for easier filtering
ben_s1_state_cara['CLM_FROM_DT'] = ben_s1_state_cara['CLM_FROM_DT'].apply(str)
ben_s1_state_cara['CLM_FROM_YR'] = ben_s1_state_cara['CLM_FROM_DT'].str.slice(0, 4)

In [7]:
# roll-up state of interest er claims to SSA county level
# only retain 2008 claims
cara_county_metrics = ben_s1_state_cara[
    (ben_s1_state_cara.HCPCS_CD_1.isin(
        ['G0380','G0381','G0382','G0383','G0384','99281','99282','99283','99284','99285'])
    ) &
    (ben_s1_state_cara.CLM_FROM_YR == '2008')
    ].groupby(['BENE_COUNTY_CD','CLM_FROM_YR'])['CLM_ID'].nunique()

cara_county_metrics = cara_county_metrics.to_frame().reset_index()

In [8]:
# rename distinct claim ID count to ER Visits
cara_county_metrics.rename(columns={ cara_county_metrics.columns[2]: "ER_VISITS" }, inplace = True)

cara_county_metrics.head()

,BENE_COUNTY_CD,CLM_FROM_YR,ER_VISITS
0,0,2008,26
1,100,2008,70
2,150,2008,31
3,160,2008,31
4,180,2008,8


In [9]:
# Count Benes and MM by SSA County
ben_county = ben_s1_state.groupby(
    ['SP_STATE_CODE', 'BENE_COUNTY_CD']).agg(
    {'DESYNPUF_ID':'nunique', 'BENE_SMI_CVRAGE_TOT_MONS': 'sum'}
).reset_index().rename(columns={'DESYNPUF_ID':'BENE_COUNT'})

In [10]:
# Merge beneficiary count / MM at county level with ER visit count at county level
ben_county = pd.merge(ben_county,cara_county_metrics,on='BENE_COUNTY_CD',how='left')

# calculate ER visits per K
ben_county['ER_K'] = round(12000*ben_county['ER_VISITS']/ben_county['BENE_SMI_CVRAGE_TOT_MONS'],0)

## 198 visits on average
print(12000*ben_county['ER_VISITS'].sum()/ben_county['BENE_SMI_CVRAGE_TOT_MONS'].sum())

198.54749912147125


In [11]:
ben_county[['SP_STATE_CODE', 'BENE_COUNTY_CD']].head()

,SP_STATE_CODE,BENE_COUNTY_CD
0,31,0
1,31,100
2,31,150
3,31,160
4,31,180


In [12]:
countyxwalk[countyxwalk['ssastate'] == 31].head()

,county,state,ssacounty,fipscounty,cbsa,cbsaname,ssastate,fipsstate
1804,ATLANTIC,NJ,31000,34001,12100.0,"Atlantic City-Hammonton, NJ",31,34
1805,BERGEN,NJ,31100,34003,35644.0,"New York-White Plains-Wayne, NY-NJ",31,34
1806,BURLINGTON,NJ,31150,34005,15804.0,"Camden, NJ",31,34
1807,CAMDEN,NJ,31160,34007,15804.0,"Camden, NJ",31,34
1808,CAPE MAY,NJ,31180,34009,36140.0,"Ocean City, NJ",31,34


In [13]:
# concatenate state and county code to be able to join to SSA to FIPS xwalk
ben_county['BENE_COUNTY_CD'] = ben_county['SP_STATE_CODE'].astype(str) + ben_county['BENE_COUNTY_CD'].astype(str)

# override county code 310 which should be 31000 after transformation
ben_county.replace('310', '31000', inplace=True)

# convert new derived county code back to int to finish transformation needed for join
ben_county['BENE_COUNTY_CD'] = ben_county['BENE_COUNTY_CD'].astype(str).astype(int)

ben_county[['SP_STATE_CODE', 'BENE_COUNTY_CD']].head()

,SP_STATE_CODE,BENE_COUNTY_CD
0,31,31000
1,31,31100
2,31,31150
3,31,31160
4,31,31180


In [14]:
# join summarized df with SSA to FIPS xwalk
ben_county = pd.merge(ben_county,countyxwalk,how = 'left', left_on=['BENE_COUNTY_CD'], right_on = ['ssacounty'])

In [15]:
ben_county.to_csv("data\\NJ_ER_County_2008.csv", index = False)

In [16]:
print((time.time() - start_time)/60, "minutes")

2.9363151868184407 minutes
